In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import matplotlib.pyplot as plt
import torch
from torch import nn
from  typing import Callable
from functools import partial

In [3]:
TOTAL_TIME = 6.0

In [4]:
class PINN(nn.Module):
    """Simple neural network accepting two features as input and returning a single output

    In the context of PINNs, the neural network is used as universal function approximator
    to approximate the solution of the differential equation

    num_hidden: number of hiddent layers (in addition to input and output layers)
    dim_hidden: number of neurons in each hidden layer
    """
    def __init__(self, num_hidden: int, dim_hidden: int, act=nn.Tanh()):
        super(PINN, self).__init__()

        # input t into the network
        self.layer_in = nn.Linear(1, dim_hidden)
        self.layer_out = nn.Linear(dim_hidden, 1)

        num_middle = num_hidden - 1
        self.middle_layers = nn.ModuleList(
            [nn.Linear(dim_hidden, dim_hidden) for _ in range(num_middle)]
        )
        self.act = act

    def forward(self, t):
        t = t.reshape(-1, 1)
        out = self.act(self.layer_in(t))
        for layer in self.middle_layers:
            out = self.act(layer(out))
        return self.layer_out(out)

In [5]:
def f(arr: torch.tensor, t: torch.Tensor) -> torch.Tensor:
    """Compute the value of the approximate solution from the NN model"""
    return arr


def df(output: torch.Tensor, input: torch.Tensor, order: int = 1) -> torch.Tensor:
    """Compute neural network derivative with respect to input features using PyTorch autograd engine"""
    df_value = output
    for _ in range(order):
        df_value = torch.autograd.grad(
            df_value,
            input,
            grad_outputs=torch.ones_like(df_value),
            create_graph=True,
        )[0]

    return df_value


def dfdt(dep_var, t: torch.Tensor, order: int=1):
    """Derivative with respect to the time variable of arbitrary order"""
    return df(dep_var, t, order=order)

In [6]:
tt = torch.tensor
t = tt([1.,2.,3.], requires_grad=True)
t1 = t[1:]
# u = torch.sum(t*t)
u = torch.sum(t1*t1)
dfdt(u,t1)

tensor([4., 6.], grad_fn=<AddBackward0>)

In [9]:
def compute_loss(
    nn_approximator: int, x: torch.Tensor=None, t: torch.Tensor=None
) -> torch.float:
    """Compute the full loss function as interior loss + boundary loss

    This custom loss function is fully defined with differentiable tensors therefore
    the .backward() method can be applied to it
    """

    # PDE residual
    u   = nn_approximator(t) #f(nn_approximator, t)
    # u = torch.sum((t*t)[1:])
    # print("shape t[1:]: ", t[1:].shape)
    # print("shape u: ", u.shape)
    # return -1.
    interior_loss = dfdt(u,t).reshape(-1,1) + u

    u_initial = torch.tensor([1.])  # initial solution
    initial_loss_f = u[0,0] - u_initial

    # obtain the final MSE loss by averaging each loss term and summing them up
    final_loss = interior_loss.pow(2).mean()  \
               + initial_loss_f.pow(2).mean() 

    return final_loss

In [18]:
def train_model(
    nn_approximator: PINN,
    loss_fn: Callable,
    learning_rate: int = 0.001,
    max_epochs: int = 10, 
    nt=20,
    t_domain=[0., 1.]
    ) -> PINN:
    
    losses = []

    optimizer = torch.optim.Adam(nn_approximator.parameters(), lr=learning_rate, weight_decay=0.005)
    #optimizer = torch.optim.SGD(nn_approximator.parameters(), lr=learning_rate)
    for epoch in range(max_epochs):

        #t = torch.linspace(t_domain[0], t_domain[1], steps=nt, requires_grad=True)
        # Different set of t every epoch
        t = torch.rand(nt, requires_grad=True) * (t_domain[1] - t_domain[0]) + t_domain[0]
        t[0] = 0.0
        loss_fn = partial(compute_loss, t=t)

        try:
            optimizer.zero_grad()
            loss: torch.Tensor = loss_fn(nn_approximator)
            losses.append(loss.detach())
            loss.backward()  # was not in orig
            optimizer.step()

            #if epoch % 10 == 0:
                #print(f"Epoch: {epoch} - Loss: {float(loss):>7f}")

        # If type ctrl-C on the keyboard, this loop is exited, and then the program continues
        except KeyboardInterrupt:
            break

    return nn_approximator, losses

In [12]:
def plot_solution(nn_trained: PINN, t: torch.Tensor, losses):
    fig, axes = plt.subplots(1, 2, figsize=(10,4))
    axes = axes.reshape(-1)
    ax = axes[0]
    ax.clear()
    
    # Compoute exact solution at 100 points as a curve
    exact_sol = torch.exp(-t.detach())
    ax.scatter(t.detach().reshape(-1), nn_trained(t).detach().reshape(-1), label='approx')
    ax.scatter(t.detach().reshape(-1), exact_sol, label='exact')
    ax.legend()
    
    ax = axes[1]
    ax.clear()
    ax.plot(losses)
    ax.set_title("loss10")
    ax.set_xlabel("Epoch")
    ax.set_ylabel("$log_{10} (loss)$")
    ax.set_yscale("log")
    ax.grid(True)
    plt.tight_layout()
    plt.show()

In [19]:
@interact(nb_epochs=200, lr=[.001,.01,.1])
def execute_run(nb_epochs, lr, tmax=6.0, nb_hid_layers=3, pts_per_layer=30, nb_points=20, is_random=True):
    t_domain = [0.0, tmax];
    nt = nb_points

    # Every iteration, I should choose a random selection of t
    t_raw = torch.linspace(0., tmax, steps=nt, requires_grad=True)
    t = t_raw

    args = {"num_hidden": nb_hid_layers, "dim_hidden": pts_per_layer}
    nn_approximator = PINN(**args)

    # Check that autograd derivatives match finite-difference derivatives
    # assert check_gradient(nn_approximator, x, t)

    compute_loss(nn_approximator, t=t)

    # train the PINN
    loss_fn = partial(compute_loss, t=t)
    nn_approximator_trained, losses = train_model(
        nn_approximator, loss_fn=loss_fn, learning_rate=lr, max_epochs=nb_epochs, nt=nt, t_domain=t_domain
    )
    plot_solution(nn_approximator_trained, t, losses)

interactive(children=(IntSlider(value=200, description='nb_epochs', max=600, min=-200), Dropdown(description='…